In [1]:
import pandas as pd 
import numpy as np 
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, make_scorer
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import RepeatedKFold, KFold, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.pipeline import Pipeline

In [2]:
# LEER LOS DATOS 
file_in = pd.read_csv('dead_due_to_AHE.csv') #lee archivo 
file_in = file_in.drop(['subject_id', 'hadm_id', 'icustay_id'], axis= 'columns') #elimino las columnas que no ocupo 

# IMPUTACIÓN DE DATOS 
x = file_in.drop(['dead'], axis='columns') #para mis variables de entrada soalamente elimino la varaible a predecir
y= file_in.dead #me quedo con la variable a predecir en y 
columnas= x.columns 


# DIVIDIR CONJUNTO DE ENTRENAMIENTO, VALIDACIÓN Y PRUEBA 
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.3, )
print('X_train:', x_train.shape, 'X_test:', x_test.shape) 

#data = x.replace(" ", np.NaN)
#values = data.values
#imp = IterativeImputer(max_iter=10, random_state=0)
imp = KNNImputer(n_neighbors=5, weights= 'uniform').fit(x_train)
##imp = MissForest()
#imp= SimpleImputer(missing_values=np.nan, strategy='most_frequent').fit(x_train) #mean, most frequent, constant 
x_train = imp.fit_transform(x_train)
x_test = imp.fit_transform(x_test)
print('X_train:', x_train.shape, 'X_test:', x_test.shape)

X_train: (3563, 40) X_test: (1528, 40)
X_train: (3563, 40) X_test: (1528, 40)


In [3]:
print(type(x_train))
x_train_imputado = pd.DataFrame(x_train, columns=[columnas]) #convirtiendo el arreglo en dataframe 
x_test_imputado = pd.DataFrame(x_test, columns=[columnas])
x_train_imputado

<class 'numpy.ndarray'>


,age,kg,creatinine,sexo,first_sicu,first_micu,first_ccu,first_tsicu,first_csru,first_nicu,...,resprate_max,tempc_min,tempc_mean,tempc_max,spo2_min,spo2_mean,spo2_max,pao2_min,pao2_mean,pao2_max
0,71.0,69.400002,1.7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,25.0,35.944443,37.005050,38.055556,89.0,95.181818,100.0,94.0,151.000000,306.0
1,77.0,66.000000,0.6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,35.0,36.000000,37.084230,38.500000,81.0,98.114754,100.0,159.0,295.625000,425.0
2,60.0,60.000000,1.3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,29.0,35.555556,37.105841,38.777779,93.0,98.555556,100.0,91.0,145.900000,290.0
3,42.0,50.000000,2.8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,35.0,36.000002,37.045752,38.444443,90.0,94.962963,100.0,82.6,92.200000,101.8
4,70.0,82.199997,0.6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,38.0,35.000000,36.822854,37.799999,90.0,97.797654,100.0,72.0,157.787879,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3558,67.0,52.299999,1.7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,27.0,35.444446,36.048148,37.444445,91.0,94.787879,100.0,84.0,84.000000,84.0
3559,87.0,63.000000,1.4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,32.0,36.555557,37.194444,38.111110,91.0,97.582090,100.0,239.0,239.000000,239.0
3560,76.0,85.000000,2.2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,35.0,35.333332,36.037879,36.944444,93.0,97.750000,100.0,317.0,338.333333,363.0
3561,45.0,64.599998,1.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,40.0,35.611110,37.447778,39.055557,90.0,98.676755,100.0,60.0,120.272727,387.0


In [4]:
# ESTANDARIZACIÓN DE DATOS 
estandarizar = ['age', 'kg', 'creatinine','heartrate_min', 'heartrate_mean', 'heartrate_max', 'sysbp_min',
       'sysbp_mean', 'sysbp_max', 'diasbp_min', 'diasbp_mean', 'diasbp_max',
       'meanbp_min', 'meanbp_mean', 'meanbp_max', 'resprate_min','resprate_mean', 'resprate_max', 'tempc_min', 'tempc_mean', 'tempc_max',
       'spo2_min', 'spo2_mean', 'spo2_max', 'pao2_min', 'pao2_mean','pao2_max']
dummies = ['sexo', 'first_sicu', 'first_micu',
       'first_ccu', 'first_tsicu', 'first_csru', 'first_nicu', 'last_sicu','last_micu', 'last_ccu', 'last_tsicu', 'last_csru', 'last_nicu']

#Intentar con las varaibles separadas y también juntas 
scaler = StandardScaler().fit(x_train_imputado)
#transformed_values[estandarizar] = scaler.fit_transform(transformed_values[estandarizar]) #estandarizo las variables númericas 
x_train_escalado = scaler.fit_transform(x_train_imputado)
x_test_escalado = scaler.fit_transform(x_test_imputado)

#COLOCAR PRINTS 
print('X_train:', x_train_escalado.shape, 'X_test:', x_test_escalado.shape)

X_train: (3563, 40) X_test: (1528, 40)


In [5]:
# DEFINIR MODELO A UTILIZAR 
#RF
modeloXGB = xgb.XGBClassifier(use_label_encoder=False)
modeloRF = RandomForestClassifier()
modeloSVM = svm.LinearSVC()

#DEFINIR PARÁMETROS A ITERAR

parametrosRF = {'n_estimators': range(10, 200, 10), 'criterion':['gini', 'entropy'], 'max_depth': range(1,10,1), 
              'max_features':['auto', 'sqrt', 'log2'],  'min_impurity_decrease':[1e-4, 1e-7], 
              'class_weight':['balanced', 'balanced_subsample']} # PARA RF

parametrosRFprueba = {'n_estimators': range(10, 200, 10),'max_depth': range(1,10,1),'class_weight':['balanced']}

ratio = float(np.sum(y_train == 0)) / np.sum(y_train==1)
                      
parametrosXGB = {'loss':['deviance', 'exponential'], 'criterion':['friedman_mse', 'mse', 'mae'], 'max_features':['auto', 'sqrt',
              'log2'], 'n_estimators': range(10, 200, 10), "learning_rate": [0.1,0.05],"colsample_bytree" : [0.4, 0.8],
                    "subsample" : [0.8, 0.4], "reg_alpha" : [0.5], "reg_lambda": [2],
                    "objective": ['binary:logistic'],  'max_depth': range(1,10,1), "gamma":[10],"rate_drop": [0.5, 0.3],
                    "seed": [422], "eval_metric": ['auc'],"scale_pos_weight": [ratio]}

parametrosXGBprueba = { 'n_estimators': range(10, 200, 10), "learning_rate": [0.1,0.05], 'max_depth': range(1,10,1),
                        "eval_metric": ['auc'],"scale_pos_weight": [ratio]}

parametrosSVM = {"C":np.logspace(-3,3,7), "class_weight": ['balanced'],"random_state": [422]}
parametrosSVMprueba = {'C':np.logspace(-3,3,7), 'loss':['hinge', 'squared_hinge'], 'multi_class':['ovr', 'crammer_singer']}

#ENTRENAR GRID SEARCH 
#TIPOS DE SCORE 
balanced = class_weight.compute_sample_weight('balanced', y_train)
RKF = RepeatedKFold(n_splits=5, n_repeats=2, random_state=422) #probar los CROSS VALIDATION
SKF = StratifiedKFold(n_splits=5, shuffle= True, random_state=422)
grid = GridSearchCV(estimator= modeloSVM, param_grid= parametrosSVMprueba, cv=SKF, scoring='roc_auc', verbose=2, refit= True)

In [6]:
grid.fit(x_train, y_train, sample_weight = balanced)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV] END ...............C=0.001, loss=hinge, multi_class=ovr; total time=   0.1s


C:\Users\Daniel Dimas\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Daniel Dimas\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV] END ...............C=0.001, loss=hinge, multi_class=ovr; total time=   0.1s
[CV] END ...............C=0.001, loss=hinge, multi_class=ovr; total time=   0.0s


C:\Users\Daniel Dimas\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Daniel Dimas\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV] END ...............C=0.001, loss=hinge, multi_class=ovr; total time=   0.1s
[CV] END ...............C=0.001, loss=hinge, multi_class=ovr; total time=   0.1s


...

[CV] END C=1000.0, loss=squared_hinge, multi_class=crammer_singer; total time=  57.7s


C:\Users\Daniel Dimas\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[CV] END C=1000.0, loss=squared_hinge, multi_class=crammer_singer; total time=  55.5s


C:\Users\Daniel Dimas\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=422, shuffle=True),
             estimator=LinearSVC(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'loss': ['hinge', 'squared_hinge'],
                         'multi_class': ['ovr', 'crammer_singer']},
             scoring='roc_auc', verbose=2)

In [7]:
# METRICAS DEL MODELO 
auc_train = grid.best_score_
std_train = grid.cv_results_['std_test_score'][grid.best_index_]
best_params = grid.best_params_
best_estimator = grid.best_estimator_
print("auc:",auc_train)
print("std_validation:",std_train)
print('best parameters:',best_params)
print('best estimator:', best_estimator)

auc: 0.8241831960599019
std_validation: 0.015131687987003185
best parameters: {'C': 0.01, 'loss': 'squared_hinge', 'multi_class': 'crammer_singer'}
best estimator: LinearSVC(C=0.01, multi_class='crammer_singer')
